# Interactive Demo for Metrics

* command line executables: see README.md
* algorithm documentation: [metrics.py API & Algorithm Documentation](metrics.py_API_Documentation.ipynb)
* **make sure you enabled interactive widgets via: **
```
sudo jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

* **make sure you use the correct Kernel** matching the your evo Python version (otherwise use the menu Kernel->Change..

...some modules and settings for this demo:

In [ ]:
from __future__ import print_function
import logging
import sys

# Python version we're running
print(sys.executable)
# set logging verbosity level
# DEBUG=verbose, INFO=normal, WARNING=silent --> change requires notebook Kernel restart
logging.basicConfig(format="%(message)s", stream=sys.stdout, level=logging.DEBUG)

In [ ]:
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [ ]:
# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": False, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": False, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
du_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit},
    value=Unit.frames, description='delta_unit'
)
pm_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pr_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

---

## Load trajectories

In [ ]:
from evo.tools import file_interface

**Load KITTI files** with entries of the first three rows of $\mathrm{SE}(3)$ matrices per line (no timestamps):

In [ ]:
traj_ref = file_interface.read_kitti_poses_file("test/data/KITTI_00_gt.txt")
traj_est = file_interface.read_kitti_poses_file("test/data/KITTI_00_ORB.txt")

**...or load a ROS bagfile** with `geometry_msgs/PoseStamped` topics:

In [ ]:
try:
    import rosbag
    bag_handle = rosbag.Bag("test/data/ROS_example.bag")
    traj_ref, traj_est = file_interface.load_assoc_bag_trajectories(
        bag_handle,
        ref_topic = "groundtruth",
        est_topic = "ORB-SLAM",
        max_diff = 0.01,
        offset_2 = 0.0,
    )
except ImportError as e:
    print(e)  # ROS not found

**... or load TUM files with** 3D position and orientation quaternion per line ($x$ $y$ $z$ $q_x$ $q_y$ $q_z$ $q_w$):

In [ ]:
traj_ref, traj_est = file_interface.load_assoc_tum_trajectories(
    ref_file = "test/data/fr2_desk_groundtruth.txt",
    est_file = "test/data/fr2_desk_ORB_kf_mono.txt",
    max_diff = 0.01,
    offset_2 = 0.0,
)

In [ ]:
print(traj_ref)
print(traj_est)

---

## APE

Algorithm and API explanation: [see here](metrics.py_API_Documentation.ipynb#ape_math)

### Interactive APE Demo
***Run the code below, configure the parameters in the GUI and press the update button.***

(uses the trajectories loaded above)

In [ ]:
import evo.main_ape as main_ape

# store the results here for later
stats, results = {}, {}
count = 0

def callback_ape(pose_relation, align, correct_scale, plot_mode, show_plot):
    _stats, _results = main_ape.main_ape(traj_ref, traj_est, 
                                         pose_relation=pose_relation, align=align, correct_scale=correct_scale, 
                                         show_plot=show_plot, plot_mode=plot_mode)
    global results, stats, count
    count += 1
    results["APE test #" + str(count)] = _results
    stats["APE test #" + str(count)] = _stats
    
_ = ipywidgets.interact_manual(callback_ape, pose_relation=pr_selector, plot_mode=pm_selector,
                               **{c.description: c.value for c in check_boxes_ape})

---

## RPE

Algorithm and API explanation: [see here](metrics.py_API_Documentation.ipynb#rpe_math)

### Interactive RPE Demo

***Run the code below, configure the parameters in the GUI and press the update button.***

(uses the trajectories loaded above, alignment only useful for visualization here)

In [ ]:
import evo.main_rpe as main_rpe

# store the results here for later
stats, results = {}, {}
count = 0

def callback_rpe(pose_relation, delta, delta_unit, all_pairs, align, correct_scale, plot_mode, show_plot):
    _stats, _results = main_rpe.main_rpe(traj_ref, traj_est, 
                                         pose_relation=pose_relation, delta=delta, delta_unit=delta_unit, 
                                         all_pairs=all_pairs, align=align, correct_scale=correct_scale, 
                                         show_plot=show_plot, plot_mode=plot_mode, support_loop=True)
    global results, stats, count
    count += 1
    results["RPE test #" + str(count)] = _results
    stats["RPE test #" + str(count)] = _stats

_ = ipywidgets.interact_manual(callback_rpe, pose_relation=pr_selector, plot_mode=pm_selector, 
                               delta=delta_input, delta_unit=du_selector, 
                               **{c.description: c.value for c in check_boxes_rpe})

## Extensions

Use Pandas and Seaborn to visualize the results.

(this box only works if you ran the APE/RPE box 1+ times before)

In [ ]:
import pandas as pd
import seaborn as sns
from IPython.display import display

if not stats:
    raise RuntimeError("no data available yet")

stats_df = pd.DataFrame.from_dict(stats)
results_df = pd.DataFrame.from_dict(results, orient="index").transpose()
results_df = pd.melt(results_df, value_vars=list(results_df.columns.values), var_name="estimate", value_name="error")

display(stats_df)

# bar plot of stats
exclude = stats_df.index.isin(["sse"])  # don't plot sse
stats_df[~exclude].plot(kind="barh", stacked=False)

# box plot + kernel density estimate
fig_violin = plt.figure()
sns.violinplot(x=results_df["estimate"], y=results_df["error"])

# histogram + kernel density estimate
dist_grid = sns.FacetGrid(results_df, col="estimate", col_wrap=4)
dist_grid.map(sns.distplot, "error")  # fits=stats.gamma